# 모델 불러오기

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
attend_model = tf.keras.models.load_model('nsc_1-1_based_attendModel.h5')
vent_model = tf.keras.models.load_model('nsc_1-1_based_ventModel.h5')

###데이터 불러오기

In [ ]:
classlist = ['isc_2_1','isc_3_1','isc_3_3', # 0, 1, 2
             'nsc_1_1','nsc_1_2','nsc_2_5', # 3, 4, 5 
             'sdc_1_1','sdc_2_1',           # 6, 7
             'sic_2_6','sic_4_2']           # 8, 9
# ----------------------- 데이터 선택------------------------------
NUMBER = 9;
# ----------------------- 데이터 선택------------------------------

for_predict = '_for_predict'
choosefile = classlist[NUMBER]
predictdata_address = 'https://raw.githubusercontent.com/hanyeob/fine_dust/master/csv%20files/' + choosefile + for_predict +".csv"

In [ ]:
predictdata = pd.read_csv(predictdata_address)
predictdata[0:5]

,Unnamed: 0,tt,CO2_outdoor,PM2.5_outdoor,PM10_outdoor,CO2_classroom,PM2.5_classroom,PM10_classroom,CO2_classroom_change,PM2.5_classroom_change,PM10_classroom_change
0,0,2019-12-09 17:21:58,2.172880,-0.243034,0.529866,-0.342581,1.259666,0.056165,-0.014231,-0.017581,-0.794181
1,1,2019-12-09 17:22:58,2.095391,-0.212904,0.441144,-0.344607,1.257464,-0.076304,-0.014231,-0.017581,-0.794181
2,2,2019-12-09 17:23:58,1.862921,-0.240694,0.554274,-0.346633,1.247452,-0.084361,-0.014231,-0.094764,-0.048080
3,3,2019-12-09 17:24:58,0.868470,-0.247208,0.772440,-0.350686,1.237001,-0.156891,-0.028180,-0.099106,-0.434727
4,4,2019-12-09 17:25:58,0.429361,-0.256958,0.792566,-0.352712,1.187057,0.102730,-0.014231,-0.489424,1.557173


In [ ]:
try:
  predictdata = predictdata.drop(columns=['Unnamed: 0'])
except:
  print(predictdata.columns)

## 독립변수 추출

In [ ]:
predictdata_idp = predictdata[['CO2_outdoor', 'PM2.5_outdoor', 'PM10_outdoor', 'CO2_classroom',
       'PM2.5_classroom', 'PM10_classroom', 'CO2_classroom_change',
       'PM2.5_classroom_change', 'PM10_classroom_change']]

In [ ]:
predictdata_idp.describe

<bound method NDFrame.describe of       CO2_outdoor  PM2.5_outdoor  ...  PM2.5_classroom_change  PM10_classroom_change
0        2.172880      -0.243034  ...               -0.017581              -0.794181
1        2.095391      -0.212904  ...               -0.017581              -0.794181
2        1.862921      -0.240694  ...               -0.094764              -0.048080
3        0.868470      -0.247208  ...               -0.099106              -0.434727
4        0.429361      -0.256958  ...               -0.489424               1.557173
...           ...            ...  ...                     ...                    ...
4093    -0.578006      -1.237343  ...               -0.134481               0.009822
4094    -0.539261      -1.242357  ...               -0.334531              -0.092669
4095    -0.565091      -1.241022  ...               -0.371687              -0.577431
4096    -0.745900      -1.256271  ...               -0.162493               0.060514
4097    -0.603836      -1.26570

# 예측

In [ ]:
attend_result = attend_model.predict(predictdata_idp)
vent_result = vent_model.predict(predictdata_idp)

In [ ]:
attend_result.shape

(4098, 2)

In [ ]:
attend_result = pd.DataFrame(data = np.round(attend_result), columns=['attend_No', 'attend_Yes'])
attend_result.sum()

attend_No     3188.0
attend_Yes     910.0
dtype: float32

In [ ]:
vent_result = pd.DataFrame(data = np.round(vent_result), columns=['vent_No', 'vent_Yes'])
vent_result.sum()

vent_No     3671.0
vent_Yes     427.0
dtype: float32

## 예측 데이터 통합

In [ ]:
def merge_predict(data) :
  for i in range(0,len(data)) :
    if (attend_result.loc[i,'attend_No'] == 1):
      data.loc[i,'attend'] = 0
    else :
      data.loc[i,'attend'] = 1

    if (vent_result.loc[i,'vent_No'] == 1):
      data.loc[i,'vent'] = 0
    else :
      data.loc[i,'vent'] = 1

## _for_predict 데이터에 통합 (일반화된 데이터) 

In [ ]:
merge_predict(predictdata)
predictdata.columns

Index(['tt', 'CO2_outdoor', 'PM2.5_outdoor', 'PM10_outdoor', 'CO2_classroom',
       'PM2.5_classroom', 'PM10_classroom', 'CO2_classroom_change',
       'PM2.5_classroom_change', 'PM10_classroom_change', 'attend', 'vent'],
      dtype='object')

##원본데이터에 통합(일반화 되지 않은 데이터)

In [ ]:
origindata_address = 'https://raw.githubusercontent.com/hanyeob/fine_dust/master/csv%20files/' + choosefile +'.csv'
origindata = pd.read_csv(origindata_address)
origindata[0:5]

,tt,0.3_outdoor,0.5_outdoor,1.0_outdoor,3.0_outdoor,5.0_outdoor,10.0_outdoor,0.3_classroom,0.5_classroom,1.0_classroom,3.0_classroom,5.0_classroom,10.0_classroom,temp_outdoor,RH_outdoor,AH_outside,CO2_outdoor,temp_classroom,RH_classroom,AH_classroom,CO2_classroom,ACH,temp_meteo_425,RH_meteo_425,WD_meteo_425,WS_meteo_425,temp_meteo_889,RH_meteo_889,WD_meteo_889,WS_meteo_889,PM10_air_outdoor,PM2.5_air_outdoor,temp_air_outdoor,RH_air_outdoor,AH_air_outdoor,CO2_air_outdoor,VOCS_air_outdoor,Noise_air_outdoor,PM10_air_classroom,PM2.5_air_classroom,temp_air_classroom,RH_air_classroom,AH_air_classroom,CO2_air_classroom,VOCS_air_classroom,Noise_air_classroom,ΔT
0,9-12-2019 17:21:58,370382300,67033920,3442050,163958,70318,7420,214670300,36660780,2059364,109541,63251,14841,14.0,36,NaN,673,22.0,21,NaN,572,NaN,6.2,62.3,137.4,1.2,7.1,56.7,93.2,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9-12-2019 17:22:58,374975600,68299650,3689046,176325,82686,4594,213868600,36736400,2059717,124735,51590,10247,13.7,35,NaN,667,22.0,21,NaN,571,NaN,6.4,62.3,170.4,2.8,7.2,56.5,66.2,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9-12-2019 17:23:58,371719500,67523320,3385866,152297,66431,8481,212717300,36209540,2102827,112014,54417,9894,13.3,35,NaN,649,22.0,21,NaN,570,NaN,6.4,62.2,163.2,1.1,7.2,56.1,77.5,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9-12-2019 17:24:58,370673200,67153360,3345936,145230,75265,9894,213726200,35985870,2047350,113781,57597,6360,12.5,35,NaN,572,21.9,21,NaN,568,NaN,6.4,62.2,160.0,1.3,7.1,56.3,73.4,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9-12-2019 17:25:58,366765000,66251590,3392933,190813,72792,9187,208327600,35034630,2043463,133922,71025,15194,11.7,35,NaN,538,21.9,21,NaN,567,NaN,6.5,61.9,158.3,2.0,7.1,56.7,82.6,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merge_predict(origindata)
origindata.columns

Index(['tt', '0.3_outdoor', '0.5_outdoor', '1.0_outdoor', '3.0_outdoor',
       '5.0_outdoor', '10.0_outdoor', '0.3_classroom', '0.5_classroom',
       '1.0_classroom', '3.0_classroom', '5.0_classroom', '10.0_classroom',
       'temp_outdoor', 'RH_outdoor', 'AH_outside', 'CO2_outdoor',
       'temp_classroom', 'RH_classroom', 'AH_classroom', 'CO2_classroom',
       'ACH', 'temp_meteo_425', 'RH_meteo_425', 'WD_meteo_425', 'WS_meteo_425',
       'temp_meteo_889', 'RH_meteo_889', 'WD_meteo_889', 'WS_meteo_889',
       'PM10_air_outdoor', 'PM2.5_air_outdoor', 'temp_air_outdoor',
       'RH_air_outdoor', 'AH_air_outdoor', 'CO2_air_outdoor',
       'VOCS_air_outdoor', 'Noise_air_outdoor', 'PM10_air_classroom',
       'PM2.5_air_classroom', 'temp_air_classroom', 'RH_air_classroom',
       'AH_air_classroom', 'CO2_air_classroom', 'VOCS_air_classroom',
       'Noise_air_classroom', 'ΔT', 'attend', 'vent'],
      dtype='object')

#데이터 시각화

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
CO2_fig = go.Figure()
CO2_fig = make_subplots(specs=[[{"secondary_y": True}]])

CO2_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['CO2_outdoor'], mode='lines',name='CO2_outdoor'),secondary_y=True)
CO2_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['CO2_classroom'], mode='lines',name='CO2_classroom'),secondary_y=True)
CO2_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['attend']*100,line=dict(color='rgba(250, 211, 144, 0.5)'),stackgroup='one',mode='lines',name='attend'),secondary_y=False,)
CO2_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['vent']*100,line=dict(color='rgba(184, 233, 148, 0.5)'),stackgroup='one',mode='lines',name='vent'),secondary_y=False,)

CO2_fig.update_layout(title={'text': "CO2_fig",'y':0.95,'x':0.4},title_font_size=50)
CO2_fig.update_layout(showlegend=True,xaxis_type='category',yaxis=dict(type='linear',range=[1, 100],ticksuffix='%'))
CO2_fig.update_yaxes(title_text="CO2", secondary_y=True)
CO2_fig.update_yaxes(title_text="Attend & Vent", secondary_y=False)

In [ ]:
PM25_fig = go.Figure()
PM25_fig = make_subplots(specs=[[{"secondary_y": True}]])

PM25_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['PM2.5_outdoor'], mode='lines',name='PM2.5_outdoor'),secondary_y=True)
PM25_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['PM2.5_classroom'], mode='lines',name='PM2.5_classroom'),secondary_y=True)
PM25_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['attend']*100,line=dict(color='rgba(250, 211, 144, 0.5)'),stackgroup='one',mode='lines',name='attend'),secondary_y=False,)
PM25_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['vent']*100,line=dict(color='rgba(184, 233, 148, 0.5)'),stackgroup='one',mode='lines',name='vent'),secondary_y=False,)

PM25_fig.update_layout(title={'text': "PM2.5_fig",'y':0.95,'x':0.4},title_font_size=50)
PM25_fig.update_layout(showlegend=True,xaxis_type='category',yaxis=dict(type='linear',range=[1, 100],ticksuffix='%'))
PM25_fig.update_yaxes(title_text="PM2.5", secondary_y=True)
PM25_fig.update_yaxes(title_text="Attend & Vent", secondary_y=False)

In [ ]:
PM10_fig = go.Figure()
PM10_fig = make_subplots(specs=[[{"secondary_y": True}]])

PM10_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['PM10_outdoor'], mode='lines',name='PM10_outdoor'),secondary_y=True)
PM10_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['PM10_classroom'], mode='lines',name='PM10_classroom'),secondary_y=True)
PM10_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['attend']*100,line=dict(color='rgba(250, 211, 144, 0.5)'),stackgroup='one',mode='lines',name='attend'),secondary_y=False,)
PM10_fig.add_trace(go.Scatter(x=predictdata['tt'], y=predictdata['vent']*100,line=dict(color='rgba(184, 233, 148, 0.5)'),stackgroup='one',mode='lines',name='vent'),secondary_y=False,)

PM10_fig.update_layout(title={'text': "PM10_fig",'y':0.95,'x':0.4},title_font_size=50)
PM10_fig.update_layout(showlegend=True,xaxis_type='category',yaxis=dict(type='linear',range=[1, 100],ticksuffix='%'))
PM10_fig.update_yaxes(title_text="PM10", secondary_y=True)
PM10_fig.update_yaxes(title_text="Attend & Vent", secondary_y=False)

#데이터 저장

In [ ]:
predictdata.to_csv(choosefile+for_predict+'_finished.csv',index=False)
origindata.to_csv(choosefile+'_finished.csv',index=False)